In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [2]:
with open('sentences_new.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [4]:
total_words

2964

In [5]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [6]:
input_sequences

[[4, 1576],
 [4, 1576, 70],
 [4, 1576, 70, 1978],
 [4, 1576, 70, 1978, 1979],
 [4, 1576, 70, 1978, 1979, 5],
 [4, 1576, 70, 1978, 1979, 5, 119],
 [175, 5],
 [175, 5, 6],
 [175, 5, 6, 7],
 [175, 5, 6, 7, 508],
 [175, 5, 6, 7, 508, 909],
 [4, 80],
 [4, 80, 7],
 [4, 80, 7, 342],
 [4, 80, 7, 342, 319],
 [4, 80, 7, 342, 319, 11],
 [4, 80, 7, 342, 319, 11, 70],
 [4, 80, 7, 342, 319, 11, 70, 387],
 [53, 54],
 [53, 54, 250],
 [53, 54, 250, 2],
 [53, 54, 250, 2, 509],
 [53, 54, 250, 2, 509, 5],
 [53, 54, 250, 2, 509, 5, 1343],
 [4, 29],
 [4, 29, 220],
 [4, 29, 220, 1],
 [4, 29, 220, 1, 559],
 [4, 29, 220, 1, 559, 5],
 [4, 29, 220, 1, 559, 5, 39],
 [4, 29, 220, 1, 559, 5, 39, 2],
 [4, 29, 220, 1, 559, 5, 39, 2, 66],
 [4, 29, 220, 1, 559, 5, 39, 2, 66, 458],
 [7, 150],
 [7, 150, 1],
 [7, 150, 1, 287],
 [7, 150, 1, 287, 20],
 [7, 150, 1, 287, 20, 267],
 [7, 150, 1, 287, 20, 267, 343],
 [4, 97],
 [4, 97, 1],
 [4, 97, 1, 459],
 [4, 97, 1, 459, 160],
 [4, 97, 1, 459, 160, 11],
 [4, 97, 1, 459, 160, 1

In [7]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [8]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [9]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [10]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 100)           296400    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 2964)              447564    
                                                                 
Total params: 894564 (3.41 MB)
Trainable params: 894564 (3.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)


Epoch 1/100


1421/1421 [==============================] - 48s 27ms/step - loss: 5.8184 - accuracy: 0.0856
Epoch 2/100
1421/1421 [==============================] - 39s 27ms/step - loss: 4.4609 - accuracy: 0.2208
Epoch 3/100
1421/1421 [==============================] - 38s 26ms/step - loss: 3.5989 - accuracy: 0.3286
Epoch 4/100
1421/1421 [==============================] - 36s 25ms/step - loss: 2.9968 - accuracy: 0.4050
Epoch 5/100
1421/1421 [==============================] - 35s 25ms/step - loss: 2.5618 - accuracy: 0.4619
Epoch 6/100
1421/1421 [==============================] - 38s 26ms/step - loss: 2.2313 - accuracy: 0.5063
Epoch 7/100
1421/1421 [==============================] - 39s 27ms/step - loss: 1.9688 - accuracy: 0.5456
Epoch 8/100
1421/1421 [==============================] - 39s 27ms/step - loss: 1.7534 - accuracy: 0.5825
Epoch 9/100
1421/1421 [==============================] - 46s 32ms/step - loss: 1.5743 - accuracy: 0.6137
Epoch 10/100
1421/1421 [============================

In [43]:
seed_text = "India will be"
next_words = 5

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 18ms/step
India will be proud of the upcoming community
